In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('po_data.csv', index_col='time', parse_dates=True)

In [3]:
data

,symbol_id,open,close,high,low,asset
time,,,,,,
1679382900,66,1.03843,1.03835,1.03855,1.03824,EURUSD_otc
1679382960,66,1.03836,1.03838,1.03845,1.03825,EURUSD_otc
1679383020,66,1.03839,1.03846,1.03849,1.03826,EURUSD_otc
1679383080,66,1.03847,1.03869,1.03871,1.03847,EURUSD_otc
1679383140,66,1.03868,1.03846,1.03868,1.03836,EURUSD_otc
...,...,...,...,...,...,...
1715527080,66,1.11128,1.11122,1.11138,1.11111,EURUSD_otc
1715527140,66,1.11121,1.11109,1.11121,1.11090,EURUSD_otc
1715527200,66,1.11108,1.11121,1.11149,1.11104,EURUSD_otc


In [4]:
data.drop(['asset', 'symbol_id'], axis=1, inplace=True)

In [5]:
data

,open,close,high,low
time,,,,
1679382900,1.03843,1.03835,1.03855,1.03824
1679382960,1.03836,1.03838,1.03845,1.03825
1679383020,1.03839,1.03846,1.03849,1.03826
1679383080,1.03847,1.03869,1.03871,1.03847
1679383140,1.03868,1.03846,1.03868,1.03836
...,...,...,...,...
1715527080,1.11128,1.11122,1.11138,1.11111
1715527140,1.11121,1.11109,1.11121,1.11090
1715527200,1.11108,1.11121,1.11149,1.11104


In [6]:
def rma(x, window):
    """
    Calculate an Exponential Moving Average over a series `x` with a window `window`.
    """
    return x.ewm(span=window, adjust=False).mean()

def rsi(series, period=14):
    """
    Calculate Relative Strength Index (RSI) of a given Pandas series and period length.
    """
    delta = series.diff()  # Change between prices
    up = delta.clip(lower=0)  # Only keep the positive differences (price increases)
    down = -delta.clip(upper=0)  # Only keep the negative differences, make them positive
    
    # Calculate the Exponential Moving Average (RMA) for upward and downward changes
    gain = rma(up, period)
    loss = rma(down, period)
    
    rs = gain / loss  # Relative Strength
    rsi = 100 - (100 / (1 + rs))  # Relative Strength Index
    
    return rsi

def ema(x, window):
    """
    Calculate Exponential Moving Average (EMA) similar to the TradingView Pine Script version.
    """
    return x.ewm(alpha=(2/(window+1)), adjust=False).mean()

def sma(x, window):
    """
    Calculate Simple Moving Average (SMA) using pandas.
    """
    return x.rolling(window).mean()

def macd(x, fast_length, slow_length, is_sma = False):
    fast_ma = sma(x, fast_length) if is_sma else ema(x, fast_length)
    slow_ma = sma(x, slow_length) if is_sma else ema(x, slow_length)
        
    return fast_ma - slow_ma

def macd_signal(macd, window, is_sma=False):
    return sma(macd, window) if is_sma else ema(macd, window)

def bollinger_bands(x, window, mult=2):
    basis = sma(x, window)
    
    dev = mult * x.rolling(window).std()
    
    upper = basis + dev
    lower = basis - dev
    
    return basis, upper, lower

def stochastique(x, window):
    return 100 * (x - x.rolling(window).min()) / (x.rolling(window).max() - x.rolling(window).min())
    
def stochastique_signal(x, window):
    return x.rolling(window).mean()

def sen(x, window):
    return (x.rolling(window).max() + x.rolling(window).min()) / 2

def senko_span_A(tekan_sen, kujin_sen, shift):
    return ((tekan_sen + kujin_sen) / 2).shift(shift)

def senko_span_B(x, window, shift):
    return ((x.rolling(window).min() + x.rolling(window).max()) / 2).shift(shift)

In [7]:
# bollinger bands
data['bb_basis'], data['bb_upper'], data['bb_lower'] = bollinger_bands(data[['close']], 20)

# macd
data['macd_ema'] = macd(data[['close']], 12, 26)
data['macd_sma'] = macd(data[['close']], 12, 26, True)
data['macd_signal_ema'] = macd_signal(data[['macd_ema']], 9)
data['macd_signal_sma'] = macd_signal(data[['macd_sma']], 9, True)

# rsi
data['rsi_9'] = rsi(data[['close']], 9)
data['rsi_14'] = rsi(data[['close']], 14)
data['rsi_28'] = rsi(data[['close']], 28)

# stochastique
data['stochastique'] = stochastique(data[['close']], 14)
data['stochastique_signal'] = stochastique_signal(data[['stochastique']], 14)

# ichimoku
data['tekan_sen'] = sen(data[['close']] ,9)
data['kujin_sen'] = sen(data[['close']] ,26)
data['senko_A'] = senko_span_A(data['tekan_sen'], data['kujin_sen'], 26)
data['senko_B'] = senko_span_B(data[['close']], 52, 26)

In [12]:
data['3min_later'] = data.shift(-3)['close']
data['3min_later_result'] = 'equal'
data.loc[data['3min_later'] > data['close'], '3min_later_result'] = 'buy'
data.loc[data['3min_later'] < data['close'],'3min_later_result'] = 'sell'
data

,open,close,high,low,bb_basis,bb_upper,bb_lower,macd_ema,macd_sma,macd_signal_ema,...,rsi_14,rsi_28,stochastique,stochastique_signal,tekan_sen,kujin_sen,senko_A,senko_B,3min_later,3min_later_result
time,,,,,,,,,,,,,,,,,,,,,
1679387520,1.04074,1.04069,1.04091,1.04053,1.040141,1.041177,1.039104,0.000353,0.000536,0.000285,...,60.610263,60.993022,74.400000,90.780983,1.040535,1.040020,1.039457,1.039115,1.04059,sell
1679387580,1.04070,1.04024,1.04076,1.04021,1.040187,1.041149,1.039224,0.000308,0.000501,0.000290,...,45.056456,52.373755,38.400000,86.380983,1.040535,1.040020,1.039457,1.039115,1.04053,buy
1679387640,1.04026,1.04060,1.04066,1.04018,1.040258,1.041108,1.039408,0.000298,0.000514,0.000291,...,55.578922,57.530669,64.957265,85.473675,1.040625,1.040020,1.039457,1.039115,1.04023,sell
1679387700,1.04061,1.04059,1.04078,1.04051,1.040305,1.041117,1.039493,0.000286,0.000494,0.000290,...,55.239845,57.345412,64.102564,82.909573,1.040625,1.040020,1.039422,1.039115,1.04032,sell
1679387760,1.04058,1.04053,1.04061,1.04033,1.040333,1.041135,1.039531,0.000268,0.000470,0.000286,...,53.001258,56.179573,49.473684,81.412352,1.040625,1.040055,1.039387,1.039115,1.04030,sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715526900,1.11134,1.11160,1.11171,1.11134,1.111864,1.112693,1.111034,-0.000423,-0.000603,-0.000416,...,44.851567,40.564417,41.836735,11.928658,1.111635,1.112705,1.113368,1.113305,1.11122,sell
1715526960,1.11160,1.11135,1.11173,1.11134,1.111789,1.112510,1.111068,-0.000417,-0.000570,-0.000416,...,37.745786,37.310790,17.977528,11.998192,1.111540,1.112705,1.113395,1.113205,1.11109,sell
1715527020,1.11137,1.11129,1.11137,1.11097,1.111723,1.112365,1.111081,-0.000412,-0.000509,-0.000415,...,36.159380,36.554977,11.235955,12.482657,1.111395,1.112705,1.113420,1.113170,NaN,equal


In [13]:
data.dropna(axis=0, inplace=True)
data

,open,close,high,low,bb_basis,bb_upper,bb_lower,macd_ema,macd_sma,macd_signal_ema,...,rsi_14,rsi_28,stochastique,stochastique_signal,tekan_sen,kujin_sen,senko_A,senko_B,3min_later,3min_later_result
time,,,,,,,,,,,,,,,,,,,,,
1679387520,1.04074,1.04069,1.04091,1.04053,1.040141,1.041177,1.039104,0.000353,0.000536,0.000285,...,60.610263,60.993022,74.400000,90.780983,1.040535,1.040020,1.039457,1.039115,1.04059,sell
1679387580,1.04070,1.04024,1.04076,1.04021,1.040187,1.041149,1.039224,0.000308,0.000501,0.000290,...,45.056456,52.373755,38.400000,86.380983,1.040535,1.040020,1.039457,1.039115,1.04053,buy
1679387640,1.04026,1.04060,1.04066,1.04018,1.040258,1.041108,1.039408,0.000298,0.000514,0.000291,...,55.578922,57.530669,64.957265,85.473675,1.040625,1.040020,1.039457,1.039115,1.04023,sell
1679387700,1.04061,1.04059,1.04078,1.04051,1.040305,1.041117,1.039493,0.000286,0.000494,0.000290,...,55.239845,57.345412,64.102564,82.909573,1.040625,1.040020,1.039422,1.039115,1.04032,sell
1679387760,1.04058,1.04053,1.04061,1.04033,1.040333,1.041135,1.039531,0.000268,0.000470,0.000286,...,53.001258,56.179573,49.473684,81.412352,1.040625,1.040055,1.039387,1.039115,1.04030,sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715526720,1.11146,1.11119,1.11152,1.11103,1.112116,1.113299,1.110932,-0.000459,-0.000732,-0.000395,...,22.444427,30.264194,0.000000,6.716550,1.111635,1.112705,1.113285,1.113305,1.11160,buy
1715526780,1.11120,1.11148,1.11156,1.11115,1.112016,1.113044,1.110988,-0.000450,-0.000686,-0.000406,...,37.121906,36.783697,29.591837,8.830252,1.111635,1.112705,1.113330,1.113305,1.11135,sell
1715526840,1.11148,1.11134,1.11155,1.11127,1.111934,1.112901,1.110968,-0.000449,-0.000657,-0.000414,...,33.581770,35.083028,15.306122,9.923547,1.111635,1.112705,1.113368,1.113305,1.11129,sell


In [14]:
X = data.drop(['3min_later', '3min_later_result'], axis=1)
y = data['3min_later_result']

In [39]:
X

,open,close,high,low,bb_basis,bb_upper,bb_lower,macd_ema,macd_sma,macd_signal_ema,macd_signal_sma,rsi_9,rsi_14,rsi_28,stochastique,stochastique_signal,tekan_sen,kujin_sen,senko_A,senko_B
time,,,,,,,,,,,,,,,,,,,,
1679387520,1.04074,1.04069,1.04091,1.04053,1.040141,1.041177,1.039104,0.000353,0.000536,0.000285,0.000407,56.880388,60.610263,60.993022,74.400000,90.780983,1.040535,1.040020,1.039457,1.039115
1679387580,1.04070,1.04024,1.04076,1.04021,1.040187,1.041149,1.039224,0.000308,0.000501,0.000290,0.000433,35.683468,45.056456,52.373755,38.400000,86.380983,1.040535,1.040020,1.039457,1.039115
1679387640,1.04026,1.04060,1.04066,1.04018,1.040258,1.041108,1.039408,0.000298,0.000514,0.000291,0.000456,53.144526,55.578922,57.530669,64.957265,85.473675,1.040625,1.040020,1.039457,1.039115
1679387700,1.04061,1.04059,1.04078,1.04051,1.040305,1.041117,1.039493,0.000286,0.000494,0.000290,0.000471,52.648232,55.239845,57.345412,64.102564,82.909573,1.040625,1.040020,1.039422,1.039115
1679387760,1.04058,1.04053,1.04061,1.04033,1.040333,1.041135,1.039531,0.000268,0.000470,0.000286,0.000479,49.202146,53.001258,56.179573,49.473684,81.412352,1.040625,1.040055,1.039387,1.039115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715526720,1.11146,1.11119,1.11152,1.11103,1.112116,1.113299,1.110932,-0.000459,-0.000732,-0.000395,-0.000806,16.865418,22.444427,30.264194,0.000000,6.716550,1.111635,1.112705,1.113285,1.113305
1715526780,1.11120,1.11148,1.11156,1.11115,1.112016,1.113044,1.110988,-0.000450,-0.000686,-0.000406,-0.000803,40.800950,37.121906,36.783697,29.591837,8.830252,1.111635,1.112705,1.113330,1.113305
1715526840,1.11148,1.11134,1.11155,1.11127,1.111934,1.112901,1.110968,-0.000449,-0.000657,-0.000414,-0.000788,34.761470,33.581770,35.083028,15.306122,9.923547,1.111635,1.112705,1.113368,1.113305


In [16]:
'''
X = X[X.columns[selector.get_support()]]
X
'''

'\nX = X[X.columns[selector.get_support()]]\nX\n'

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import time
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
#model = RandomForestClassifier(n_estimators=750, max_depth=70, verbose=4, n_jobs=-1)
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder

numerical_features = make_column_selector(dtype_include=np.number)

numerical_pipeline = make_pipeline(StandardScaler())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features))

y_tranformer = LabelEncoder()

model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, max_depth=70, verbose=4, n_jobs=-1))

model.fit(X_train, y_tranformer.fit_transform(y_train))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed: 14.9min


building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  87 out of 100 | elapsed: 16.4min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 16.9min finished


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025443508190>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=70, n_jobs=-1, verbose=4))])

In [20]:
model.score(X_test, y_tranformer.transform(y_test))

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done  87 out of 100 | elapsed:    1.8s remaining:    0.2s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    2.0s finished


0.6183993235834225

In [21]:
probas = model.predict_proba(X_test)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done  87 out of 100 | elapsed:    1.7s remaining:    0.2s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    1.9s finished


In [30]:
lose = 0
win = 0
threshold = 0.67
capital = 100
per_trade = 0.01
cote = 0.7
for [long, even, short, result] in np.hstack((probas, y_test.values.reshape(y_test.shape[0], 1))):
    if short > threshold:
        if result == 'buy':
            lose = lose + 1
            capital = capital - capital * per_trade
        elif result == 'sell': 
            win = win + 1
            capital = capital + capital * per_trade * cote
    if long > threshold:
        if result == 'sell':
            lose = lose + 1
            capital = capital - capital * per_trade
        elif result == 'buy': 
            win = win + 1
            capital = capital + capital * per_trade * cote
print(f'lose: {lose}; win: {win}; ratio: {win/(lose+win)}; capital: {capital}')

lose: 1896; win: 7832; ratio: 0.8050986842105263; capital: 2.8258059347047898e+17


In [32]:
(win + lose) / len(y_test) * 60

3.855574491696722

In [33]:
from joblib import dump, load
dump(model, 'model_eurusd.joblib', compress=9) 

['model_eurusd.joblib']

## Mise en situation côté serveur python

In [ ]:
model2 = load('model_eurusd.joblib') 

In [ ]:
import json

jsonData = None

with open('data.json') as file: 
    jsonData = json.load(file)

In [ ]:
jsonData

In [44]:
df = pd.DataFrame(jsonData)

df.set_index('date', inplace=True)

df.drop(['session_date', 'start_of_day'], axis=1, inplace=True)

# bollinger bands
df['bb_basis'], df['bb_upper'], df['bb_lower'] = bollinger_bands(df[['close']], 20)

# macd
df['macd_ema'] = macd(df[['close']], 12, 26)
df['macd_sma'] = macd(df[['close']], 12, 26, True)
df['macd_signal_ema'] = macd_signal(df[['macd_ema']], 9)
df['macd_signal_sma'] = macd_signal(df[['macd_ema']], 9, True)

# rsi
df['rsi_9'] = rsi(df[['close']], 9)
df['rsi_14'] = rsi(df[['close']], 14)
df['rsi_28'] = rsi(df[['close']], 28)

df.dropna(axis=0, inplace=True)

In [45]:
df

,open,high,low,close,volume,bb_basis,bb_upper,bb_lower,macd_ema,macd_sma,macd_signal_ema,macd_signal_sma,rsi_9,rsi_14,rsi_28
date,,,,,,,,,,,,,,,
2024-02-11T22:32:00Z,1.07844,1.07846,1.07844,1.07846,2,1.078421,1.078509,1.078333,0.000014,0.000036,0.000011,0.000012,64.741109,61.761420,58.289268
2024-02-11T22:35:00Z,1.07846,1.07848,1.07844,1.07848,5,1.078430,1.078501,1.078359,0.000016,0.000035,0.000012,0.000013,75.546617,68.712645,62.019256
2024-02-11T22:36:00Z,1.07848,1.07854,1.07848,1.07854,11,1.078439,1.078518,1.078360,0.000023,0.000037,0.000014,0.000014,88.622271,80.796563,70.515225
2024-02-11T22:37:00Z,1.07853,1.07858,1.07850,1.07858,13,1.078448,1.078547,1.078349,0.000030,0.000044,0.000017,0.000017,92.129396,85.195044,74.585906
2024-02-11T22:38:00Z,1.07862,1.07866,1.07862,1.07864,8,1.078460,1.078588,1.078332,0.000041,0.000051,0.000022,0.000020,95.012162,89.397971,79.210197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01T11:21:00Z,1.07834,1.07836,1.07833,1.07833,21,1.078285,1.078601,1.077968,0.000065,0.000166,0.000067,0.000079,42.624041,49.207106,53.913394
2024-04-01T11:22:00Z,1.07833,1.07836,1.07833,1.07835,14,1.078293,1.078607,1.077978,0.000061,0.000156,0.000066,0.000077,47.839473,51.970526,55.197161
2024-04-01T11:23:00Z,1.07834,1.07838,1.07831,1.07834,30,1.078304,1.078606,1.078003,0.000056,0.000137,0.000064,0.000074,45.267722,50.388925,54.383612


In [46]:
model2.predict_proba(df)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 352 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 573 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 750 out of 750 | elapsed:    3.9s finished


array([[0.472     , 0.02933333, 0.49866667],
       [0.432     , 0.03466667, 0.53333333],
       [0.424     , 0.10666667, 0.46933333],
       ...,
       [0.36      , 0.06      , 0.58      ],
       [0.44      , 0.06533333, 0.49466667],
       [0.46266667, 0.028     , 0.50933333]])